In [16]:
import os
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from PIL import Image
import torch
device = torch.device('cuda')

model = VisionEncoderDecoderModel.from_pretrained("models/checkpoint-280000")
tokenizer = AutoTokenizer.from_pretrained("models/checkpoint-280000")
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

model.to(device)

for file in os.listdir("test-page"):
    if not file.endswith((".jpg", ".png")):
        continue

    image = Image.open(os.path.join("test-page", file)).convert("RGB")

    pixel_values = (processor(image, return_tensors="pt").pixel_values).to(device)
    generated_ids = model.generate(pixel_values)
    generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text.replace(" ", ""))

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/home/dotin13/mijo/GitHub/hand-syn/tang-syn/venv/lib/python3.11/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


：华俊龙苦笑着说：文晋；这不介乎：你们
布森一个搬：外有称西个班：还是二年级·毕
免有失公道吧吧？
··李文晋笑着说：公平∶这6工作总得
有人去干吧？你说，该给谁干呢？驯谈谈一句
他们就告诉我：这是教务他的婴稚心廖但长远
说：你们一起昂向午饭龙考师·领导看得他熟
，孔贡秀粗糙稚稚；
张厚生笑着说：这叫能育多劳·你就担起
这个贪婪吧·有什么团谁·判价都令帮忙你。
··周宏义巴说：我们明翠·学松字排二作·
我们还是愉快地接受下隶·不说什么为好。当
扯假龙一个人作二羊殷忽愣，而丑丽西个班
比我们会辛苦许多：从另一方面说：也是学校
对你的贡祝·不过：无论怎么说：还是辛苦若


In [5]:
import evaluate
import numpy as np
from transformers import BasicTokenizer

cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # Predictions come with space between
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    labels_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=labels_str)
    wer = wer_metric.compute(predictions=pred_str, references=labels_str)

    return {"cer": cer, "wer": wer}

# pred = {
#     "predictions": np.array([[ 101,  101, 3330, 3152, 3232, 2218, 6624, 1092,  749,  511,  800, 1440, 6401, 1920, 2157, 6432, 8038, 1157, 2798, 2769,  794, 2445,  102]]),
#     "label_ids": np.array([[101, 3330, 3152, 3232, 2218, 1139, 3341, 749, 511, 800, 1440, 6401, 1920, 2157, 6432, 8038, 1157, 2798, 2769, 794, 2445, 102]])
# }

# compute_metrics(pred)

In [6]:
import os
from os import path

import torch
from torch.utils.data import Dataset
import torchvision.io as io
import pandas as pd
from PIL import Image

from data_aug import build_data_aug
from torch.utils.data import Subset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from transformers import default_data_collator

model = VisionEncoderDecoderModel.from_pretrained("checkpoints/checkpoint-360000/")
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")


class OCRDataset(Dataset):
    def __init__(self, dataset_dir, labels_dir, transform, processor, tokenizer, mode="train", max_target_length=32, device=None):
        self.dataset_dir = dataset_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.device = device
        self.processor = processor
        self.mode = mode
        self.max_target_length = max_target_length
        self.tokenizer = tokenizer
        self.df = self.build_df()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df["file_name"][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(path.join(self.dataset_dir, file_name)).convert("RGB")
        if self.mode == "train" and self.transform:
            image = self.transform(image)

        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        labels = self.tokenizer(text, padding="max_length",
                                stride=32,
                                truncation=True,
                                max_length=self.max_target_length).input_ids
        
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

    def build_df(self):
        li = []
        for root, dirs, files in os.walk(self.labels_dir):
            for file in files:  # Loop through the dataset tsvfiles
                if not file.endswith(".tsv"):
                    continue

                print(f"Processing {file}")
                li.append(pd.read_table(path.join(root, file),
                          names=["file_name", "text"]))

        return pd.concat(li, axis=0, ignore_index=True)
    

dataset_dir = 'dataset/data'
max_length = 64

# Define the number of samples to keep in eval dataset
num_samples = 200

eval_dataset = OCRDataset(
    dataset_dir=dataset_dir,
    labels_dir="dataset/labels/test",
    tokenizer=tokenizer,
    processor=processor,
    mode="eval",
    transform=None,
    max_target_length=max_length
)

# Create a random subset of the dataset
subset_indices = torch.randperm(len(eval_dataset))[:num_samples]
eval_dataset = Subset(eval_dataset, subset_indices.tolist())

print("Number of validation examples:", len(eval_dataset))


def init_trainer(model, tokenizer, compute_metrics, train_dataset,
                 eval_dataset):
    training_args = Seq2SeqTrainingArguments(
        predict_with_generate=True,
        evaluation_strategy="steps",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        fp16=True,
        learning_rate=4e-5,
        output_dir="./checkpoints",
        logging_dir="./logs",
        logging_strategy="steps",
        logging_steps=100,
        save_strategy="steps",
        save_total_limit=5,
        save_steps=10000,
        eval_steps=10000,
        resume_from_checkpoint="./checkpoints/",
        dataloader_num_workers=10)

    # instantiate trainer
    return Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=default_data_collator,
    )

trainer = init_trainer(model, tokenizer, compute_metrics, eval_dataset, eval_dataset)

eval_result = None
with torch.no_grad():
    eval_result = trainer.evaluate(eval_dataset, max_length=64)

/home/dotin13/mijo/GitHub/hand-syn/tang-syn/venv/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/dotin13/mijo/GitHub/hand-syn/tang-syn/venv/lib/python3.11/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to u

Processing hwdb2.0_4k.tsv
Processing hwdb_ic13_3k.tsv
Processing hw_chinese_17k.tsv
Processing hwdb2.2_3k.tsv
Processing hwdb2.1_3k.tsv
Processing hwdb_ic13_val_5k.tsv
Number of validation examples: 200


/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory
/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory


In [7]:
eval_result

{'eval_loss': 0.3815649747848511,
 'eval_cer': 0.09589737076082815,
 'eval_wer': 0.16901408450704225,
 'eval_runtime': 23.1828,
 'eval_samples_per_second': 8.627,
 'eval_steps_per_second': 0.561}

In [1]:
import torch  
from torchvision.models import efficientnet_b0, vit_l_16, densenet161, regnet_y_1_6gf
from pytorch_benchmark import benchmark

print("################################################################")
model = efficientnet_b0().cuda()
sample = torch.randn(64, 3, 224, 224)  # (B, C, H, W)
results = benchmark(model, sample, num_runs=10)

print("################################################################")
model2 =  densenet161().cuda()
sample2 = torch.randn(64, 3, 224, 224)  # (B, C, H, W)
results2 = benchmark(model2, sample2, num_runs=10)

print("################################################################")
model3 =  regnet_y_1_6gf().cuda()
sample3 = torch.randn(64, 3, 224, 224)  # (B, C, H, W)
results3 = benchmark(model3, sample3, num_runs=10)


################################################################


/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory
/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory
Warming up with batch_size=1: 100%|██████████| 1/1 [00:00<00:00,  2.71it/s]
Unable to measure model FLOPs due to error: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor
Measuring inference for batch_size=1: 100%|██████████| 10/10 [00:00<00:00, 231.46it/s]
Unable to measure energy consumption. Device must be a NVIDIA Jetson.
Measuring inference for batch_size=64: 100%|██████████| 10/10 [00:00<00:00, 27.00it/s]
Unable to measure energy consumption. Device must be a NVIDIA Jetson.


################################################################


Warming up with batch_size=1: 100%|██████████| 1/1 [00:00<00:00,  1.49it/s]
Unable to measure model FLOPs due to error: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor
Measuring inference for batch_size=1: 100%|██████████| 10/10 [00:00<00:00, 99.21it/s]
Unable to measure energy consumption. Device must be a NVIDIA Jetson.
Measuring inference for batch_size=64: 100%|██████████| 10/10 [00:01<00:00,  6.92it/s]
Unable to measure energy consumption. Device must be a NVIDIA Jetson.


################################################################


Warming up with batch_size=1: 100%|██████████| 1/1 [00:00<00:00,  3.55it/s]
Unable to measure model FLOPs due to error: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor
Measuring inference for batch_size=1: 100%|██████████| 10/10 [00:00<00:00, 124.15it/s]
Unable to measure energy consumption. Device must be a NVIDIA Jetson.
Measuring inference for batch_size=64: 100%|██████████| 10/10 [00:00<00:00, 27.06it/s]
Unable to measure energy consumption. Device must be a NVIDIA Jetson.


In [3]:
import pstats
from pstats import SortKey
p = pstats.Stats('0.log')
p.strip_dirs().sort_stats("cumtime").print_stats()

Mon Jun 19 13:33:30 2023    0.log

         2978874 function calls (2955656 primitive calls) in 38.194 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    515/1    0.001    0.000   38.194   38.194 {built-in method builtins.exec}
        1    0.059    0.059   38.194   38.194 tang_syn.py:1(<module>)
     1000    0.018    0.000   37.541    0.038 tang_syn.py:227(synthesize)
     1000    0.193    0.000   35.196    0.035 tang_syn.py:219(apply_elastic_transform)
     1000    1.034    0.001   35.002    0.035 tang_syn.py:41(elastic_transform)
     2000    0.013    0.000   24.842    0.012 _filters.py:279(gaussian_filter)
     6000    0.011    0.000   24.730    0.004 _filters.py:211(gaussian_filter1d)
     6000    0.021    0.000   24.650    0.004 _filters.py:84(correlate1d)
     6000   24.612    0.004   24.612    0.004 {built-in method scipy.ndimage._nd_image.correlate1d}
     1000    0.006    0.000    7.451    0.007 _interpolation.

In [5]:
# Cleanup fonts that does not support cjk

import os
import pygame
import pygame.freetype

pygame.freetype.init()

for file in os.listdir("fonts"):
    if file.lower().endswith((".ttf", ".otf")):
        font =  pygame.freetype.Font(f"fonts/{file}", size=10)
        metrics = font.get_metrics("你是我的小苹果，我是你的大苹果！")

        if not all(metrics):
            print(file)
            os.remove(f"fonts/{file}")